In [1]:
    #imgThresholded = cv2.adaptiveThreshold(filtered, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            #cv2.THRESH_BINARY, 199, 5)
    #cv2.imshow('Image', imgThresholded)
    #cv2.waitKey(0)

In [ ]:
#cv2.imshow('Image', imgFiltered)
#cv2.waitKey(0)

In [1]:
def extractSymbols(imgOrig, showSteps = False):
    debugImgSteps = []
    imgGray = cv2.cvtColor(imgOrig,cv2.COLOR_BGR2GRAY)
    imgFiltered = cv2.medianBlur(imgGray, 5)
    debugImgSteps.append(imgFiltered)
    
    imgCanny = cv2.Canny(imgFiltered, 50,180)
    debugImgSteps.append(imgCanny)

    kernel = np.ones((5,5), np.uint8)
    imgDilated = cv2.dilate(imgCanny, kernel, iterations=5)
    imgInvDilated = cv2.bitwise_not(cv2.dilate(imgCanny, kernel, iterations=1))
    debugImgSteps.append(imgDilated)
    debugImgSteps.append(imgInvDilated)

    imgWhiteBg = imgGray | imgInvDilated
    debugImgSteps.append(imgWhiteBg)

    contours, _ = cv2.findContours(imgDilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    boundingBoxes = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        boundingBoxes.append((x,y,w,h))

    global rowsG
    rowsG, _, _ = imgOrig.shape
    key_leftRightTopBottom = cmp_to_key(leftRightTopBottom)
    boundingBoxes = sorted(boundingBoxes, key=key_leftRightTopBottom)

    symbols = []
    for (i, box) in enumerate(boundingBoxes):
        x,y,w,h = box
        mathSymbol = imgOrig[y:y+h, x:x+w]
        h, w, _ = mathSymbol.shape
        mask = np.zeros((h+2, w+2), np.uint8)
        mathSymbol = cv2.cvtColor(mathSymbol,cv2.COLOR_BGR2GRAY)
        cv2.floodFill(mathSymbol, mask, (0,0), 255)
        symbols.append(mathSymbol)
        debugImgSteps.append(mathSymbol)
        image_name= "output_shape_number_" + str(i+1) + ".jpg"
        cv2.imwrite(image_name, mathSymbol)

    if showSteps:
        for imgStep in debugImgSteps:
            cv2.imshow('Image', imgStep)
            cv2.waitKey(0)
        else:
            cv2.destroyAllWindows()

    return symbols
        

In [ ]:
def extractSymbols(imgOrig, showSteps = False):
    debugImgSteps = []
    imgGray = cv2.cvtColor(imgOrig,cv2.COLOR_BGR2GRAY)
    modeIntensity = np.bincount(imgGray.flatten()).argmax()
    imgGray[(imgGray>=modeIntensity-30) & (imgGray<=modeIntensity+30)] = 255 #attempting to make bg white and rest non white
    imgGray[imgGray != 255] = 0
    imgFiltered = cv2.medianBlur(imgGray, 5)
    debugImgSteps.append(imgFiltered)
    
    imgCanny = cv2.Canny(imgFiltered, 50,180)
    debugImgSteps.append(imgCanny)

    kernel = np.ones((5,5), np.uint8)
    imgDilated = cv2.dilate(imgCanny, kernel, iterations=5)
    debugImgSteps.append(imgDilated)

    contours, _ = cv2.findContours(imgDilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    boundingBoxes = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        boundingBoxes.append((x,y,w,h))

    global rowsG
    rowsG, _, _ = imgOrig.shape
    key_leftRightTopBottom = cmp_to_key(leftRightTopBottom)
    boundingBoxes = sorted(boundingBoxes, key=key_leftRightTopBottom)

    symbols = []
    for (i, box) in enumerate(boundingBoxes):
        x,y,w,h = box
        mathSymbol = imgGray[y:y+h, x:x+w]
        symbols.append(mathSymbol)
        debugImgSteps.append(mathSymbol)
        image_name= "output_shape_number_" + str(i+1) + ".jpg"
        cv2.imwrite(image_name, mathSymbol)

    if showSteps:
        for imgStep in debugImgSteps:
            cv2.imshow('Image', imgStep)
            cv2.waitKey(0)
        else:
            cv2.destroyAllWindows()

    return symbols
        